Here are balanced classes, so the usual accuracy metric is used:
**Accuracy = True predictions / All predictions**
The baselines are calculated as follows:
1. Sample negative edges. Concatenate node features into edge features. Train Gradient boosting on edges features. Calculate score.
2. Create a graph on existing edges. Train Laplacian Eigenmaps. Sample negative edges. Concatenate node embeddings into edge embeddings. Train Gradient boosting on edge embeddings. Calculate score.
3. Create a graph using existing edges. Sample negative edges. Train a neural network consisting of GCN for node emedding and MLP for link prediction that maps Hadamart product edge embeddings into probability of edge existing. Calculate score.

Baseline for grade 4: beat a minimum score
Baseline for grade 6: beat a mean score